Packages

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint
import random
import math

Constants

In [2]:
g_L = 0.003
g_K = 0.36
g_Na = 1.2

In [3]:
E_L = -54.387
E_K = -77.0
E_Na = 55.0
E_Cl = -65.0

In [4]:
Cm = 10.0

In [5]:
K_B = 500
K_A = 128
gam_B = 10
gam_A = 8

Initial state

In [6]:
Vm_0 = -65.0

Equations for m, n и h:


In [7]:
def alpha_n(V):
    return (0.01 * (V + 55)) / (1 - np.exp(-0.1 * (V + 55)))

def beta_n(V):
    return 0.125 * np.exp(-0.0125 * (V + 65))

def alpha_m(V):
    return (0.1 * (V + 40)) / (1 - np.exp(-0.1 * (V + 40)))

def beta_m(V):
    return 4 * np.exp(-0.0556 * (V + 65))

def alpha_h(V):
    return 0.07 * np.exp(-0.05 * (V + 65))

def beta_h(V):
    return 1.0 / (1 + np.exp(-0.1 * (V + 35)))

def n_inf(V=Vm_0):
    return alpha_n(V) / (alpha_n(V) + beta_n(V))

def m_inf(V=Vm_0):
    return alpha_m(V) / (alpha_m(V) + beta_m(V))

def h_inf(V=Vm_0):
    return alpha_h(V) / (alpha_h(V) + beta_h(V))

Derivatives

In [8]:
def compute_derivatives(y, t0):
    dy = np.zeros((4,))
    
    V = y[0]
    n = y[1]
    m = y[2]
    h = y[3]
    
    G_L = (g_L / Cm * 1000)
    G_K = (g_K / Cm * 1000) * np.power(n, 4.0)
    G_Na = (g_Na / Cm * 1000) * np.power(m, 3.0) * h

    I_A = N_A * (1 / (1 + K_A * V_A / (L(t0, "A")))) * (V - E_Cl) * gam_A
    I_B = N_B * (1 / (1 + K_B * V_B / (L(t0, "B")))) * (V - E_Na) * gam_B
    
    dy[0] =  (I_A + I_B) 1 / Cm * 10 - (G_K * (V - E_K)) - (G_Na * (V - E_Na))
    
    dy[1] = (alpha_n(V) * (1.0 - n)) - (beta_n(V) * n)
    
    dy[2] = (alpha_m(V) * (1.0 - m)) - (beta_m(V) * m)
    
    dy[3] = (alpha_h(V) * (1.0 - h)) - (beta_h(V) * h)
    
    return dy

Initial state

In [9]:
Y = np.array([Vm_0, n_inf(), m_inf(), h_inf()])

First simulation - definition of stimulus and lenghts

In [10]:
def L(t0, syn):
  if syn == "A":
    return L_A_d.get(float(t0))
  elif syn == "B":
    return L_B_d.get(float(t0))

Computation

In [11]:
T = np.linspace(0.0, 60.0, 10000)

In [12]:
L_inc_A = 3
L_inc_B = 3
N_A = 1
V_A = 5
N_B = 1
V_B = 5
un = 0.5
ps = 3
L_out_A = 2
L_out_B = 2

In [13]:
L_A = np.zeros(len(T))
for i in range(1, len(T)):
    is_spiking = np.random.poisson(ps) > ps
    L_A[i] = max(L_A[i - 1] + int(is_spiking) * L_inc_A - L_out_A, 0.0)
L_A_d = dict(zip(T, L_A))

L_B = np.zeros(len(T))
for i in range(1, len(T)):
    is_spiking = np.random.uniform(0,1) > un
    L_B[i] = max(L_B[i - 1] + int(is_spiking) * L_inc_B - L_out_B, 0.0)
L_B_d = dict(zip(T, L_B))

In [14]:
Ey = odeint(compute_derivatives, Y, T)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  


TypeError: ignored

Graphs

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize = (20,10))
ax1.plot(T, Ey[:, 0])
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Vm (mV)')
ax1.set_title('Vm from time')
plt.grid()

ax2.plot(T, Ey[:, 1], label = 'n')
ax2.plot(T, Ey[:, 2], label = 'm')
ax2.plot(T, Ey[:, 3], label = 'h')
ax2.legend()
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel('n, m, h')
ax2.set_title('n, h, m from time')
plt.grid()
